In [2]:
import numpy
import sycomore
from sycomore.units import *
import vtk
import vtk.numpy_interface.dataset_adapter

species = sycomore.Species(
    1000*ms, 100*ms, numpy.diag([4000, 1500, 10])*um**2/s)

TE = 50*ms
G = 50*mT/m
delta = 18*ms
# Since b = (γGδ)^2 ⋅ (Δ-δ/3) and Δ = TE-δ, the b-value is:
print(
    "b={:.0f} s/mm²".format(
        ((sycomore.gamma*G*delta)**2 * (TE-4/3*delta)).convert_to(s/mm**2)))

def dw_se(species, TE, G, delta):
    model = sycomore.epg.Discrete3D(species)

    # Excitation
    model.apply_pulse(90*deg)

    # Diffusion-sensitization gradient and mixing time
    model.apply_time_interval(delta, G)
    model.apply_time_interval(TE/2-delta)

    # Refocussing
    model.apply_pulse(180*deg)

    # Mixing time and diffusion-sensitization gradient
    model.apply_time_interval(TE/2-delta)
    model.apply_time_interval(delta, G)

    return numpy.abs(model.echo)

# Generate a unit sphere
sphere_source = vtk.vtkSphereSource()
sphere_source.SetRadius(1)
sphere_source.SetThetaResolution(50)
sphere_source.SetPhiResolution(50)
sphere_source.Update()
sphere = vtk.numpy_interface.dataset_adapter.WrapDataObject(
    sphere_source.GetOutput())
print(len(sphere.Points), "points on sphere")

# Simulate the sequence with a diffusion gradient along each point on the sphere
S_0 = dw_se(species, TE, [0*mT/m, 0*mT/m, 0*mT/m], delta)
attenuation = numpy.zeros((len(sphere.Points)))
for index, direction in enumerate(sphere.Points):
    S = dw_se(species, TE, direction*G, delta)
    attenuation[index] = S/S_0
    sphere.Points[index] *= attenuation[index]
sphere.PointData.append(1-attenuation, "Attenuation")
sphere.PointData.SetActiveScalars("Attenuation")

# Recompute the normals of the surface
normals = vtk.vtkPolyDataNormals()
normals.SetInputData(sphere.VTKObject)

# Plot the result
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(normals.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(1, 1, 1)
renderer.GetActiveCamera().SetPosition(4, -0.7, -0.3)
renderer.GetActiveCamera().SetViewUp(0.2, 0.9, 0.4)

window = vtk.vtkRenderWindow()
window.AddRenderer(renderer)
window.SetSize(800, 800)
window.SetOffScreenRendering(True)

to_image = vtk.vtkWindowToImageFilter()
to_image.SetInput(window)
png_writer = vtk.vtkPNGWriter()
png_writer.SetInputConnection(to_image.GetOutputPort())
png_writer.SetFileName("anisotropic_diffusion.png")
png_writer.Write()

b=1507 s/mm²
2402 points on sphere


In [4]:
S_0

0.6065306597126334